<html>
    <body>
            <p> We will first start by importing the libraries required and import our dataset. The data may have some empty fields and those will need to be removed to get a better model. The dataset used is the following : <a href="https://www.kaggle.com/datasets/jruvika/fake-news-detection"> fake_news.csv </a></p>
    </body>
</html>

In [ ]:
import pandas as pd
import nltk
import numpy as np

In [ ]:
#save data is dataframe
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
#remove extra columns
df = df[['Headline' , 'Body' , 'Label']].copy()
df.head()

In [ ]:
#remove empty rows
df = df.dropna()

In [ ]:
#download wordnet
nltk.download('wordnet')

In [ ]:
#initialize the tokenizer
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [ ]:
#function for getting tokens
def get_words(text):
    #make tokens of everything in the text
    tokens = tokenizer.tokenize(text)
    
    #lemmatize and remove stop words here in future
    
    return tokens

In [ ]:
#test the function on a sentence 
get_words("email healthcare reform to make america great")

In [ ]:
#split 80:20 for training and testing
#find 80th percent of the length of the dataframe
split = int(len(df) * 0.8)

#split everything before that into train and the rest into test
train_df, test_df = df[:split], df[split:]

In [ ]:
#Create dictionary which stores each token and its count
# Empty dictionary for storing the tokens
token_count = {}


# Tokenize the text
for text in train_df['Body']:
    tokens = get_words(text)
    
    for token in tokens:
        if token in token_count:
            token_count[token] += 1
        else:
            token_count[token] = 1

# Print the count of unique tokens
print(len(token_count))

In [ ]:
#function to get the tokens with the count higher than specified number
def common_tokens(token, number):
    if token not in token_count:
        return False
    else:
        tokens = token_count[token] > number
        return tokens

In [ ]:
#test it
common_tokens("government" , 50)

In [ ]:
#Add the returned words in set
common = set()

#loops through the token count dictionary 
for token in token_count:
    #if the value if greater than 700 adds it to ser
    if common_tokens(token,700):
        common.add(token)
        
print(common)

In [ ]:
#give them an index from most to least appeared
#convert to list
common = list(common)

#map them
mapping = {t:i for t,i in zip(common, range(len(common)))}
print(mapping)

In [ ]:
#make bag of words
def bag_of_words(text):
    
    #make an empty vector of the length of common list
    count_vector = np.zeros(len(common))
    
    #get the tokens list and store them
    proc_tokens = get_words(text)
    
    #count them and add them to the vector
    for token in proc_tokens:
        #if not in common, ignore the tokens 
        if token not in common:
            continue
        #count the values
        index = mapping[token]
        count_vector[index] += 1
        
    return count_vector

In [ ]:
#function to get the label and text and apply bag of words function to it
def df_xy(dataframe):
    y = dataframe['Label'].to_numpy().astype(int)
    
    text = dataframe['Body']
    vectors = []
    
    for t in text:
        vector = bag_of_words(t)
        vectors.append(vector)
        
    x = np.array(vectors).astype(int)
    
    return x,y

In [ ]:
#train and test using previous function
x_train, y_train = df_xy(train_df)
x_test, y_test = df_xy(test_df)


x_train.shape , y_train.shape , x_test.shape , y_test.shape